In [ ]:
pip install selenium

In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
import pywhatkit as kit
from datetime import datetime

# Configuração do WebDriver
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# URL e login
url = ''
driver.get(url)
driver.implicitly_wait(10)

driver.find_element(By.ID, "mat-input-0").send_keys("")
driver.find_element(By.ID, "mat-input-1").send_keys("")
driver.find_element(By.ID, "mat-input-2").send_keys("")

time.sleep(5)

acessar_button = driver.find_element(By.XPATH, "//button[i[contains(@class, 'fa-sign-in')] and span[text()='Acessar']]")
acessar_button.click()

# Navegação para o Quadro de Avisos
gestao = driver.find_element(By.XPATH, "//a[span[contains(text(), '')]]")
gestao.click()

mapa = driver.find_element(By.XPATH, "//a[span[contains(text(), '')]]")
mapa.click()

# Configuração de data e hora
data_saida = driver.find_element(By.ID, 'mat-input-5')
now = datetime.now()
date_str = now.strftime('%d/%m/%Y')
driver.execute_script(f"arguments[0].value = '{date_str}';", data_saida)

horario_saida = driver.find_element(By.ID, 'mat-input-3')
time_str = '06:00:00'
driver.execute_script(f"arguments[0].value = '{time_str}';", horario_saida)

# Seleção de veículos e filtros
selecionar = driver.find_element(By.XPATH, "//button[span/i[contains(@class, '')]]")
selecionar.click()

checkbox = driver.find_element(By.XPATH, "//input[@type='checkbox' and contains(@class, '')]")
checkbox.click()

botao_selecionar_veiculos = driver.find_element(By.CSS_SELECTOR, "button.platform-flat-button span.mat-button-wrapper")
botao_selecionar_veiculos.click()

# Seleção de tipos de avisos
span_element = driver.find_element(By.XPATH, "//span[contains(@class, 'mat-select-placeholder')]")
span_element.click()

checkbox_element = driver.find_element(By.XPATH, "//span[text()=' ']/preceding-sibling::mat-pseudo-checkbox")  
checkbox_element.click()

checkbox_element = driver.find_element(By.XPATH, "//span[text()=' ']/preceding-sibling::mat-pseudo-checkbox")
checkbox_element.click()

x_offset = 800
y_offset = 200
actions = ActionChains(driver)
actions.move_by_offset(x_offset, y_offset).click().perform()

span_element = driver.find_element(By.XPATH, "//span[contains(@class, '')]")
span_element.click()

checkbox_element = driver.find_element(By.XPATH, "//span[text()='']/preceding-sibling::mat-pseudo-checkbox")  
checkbox_element.click()

# Seleção adicional de avisos
element = driver.find_element(By.XPATH, "//span[text()='']")
checkbox = element.find_element(By.XPATH, "./preceding-sibling::mat-pseudo-checkbox")
checkbox.click()

element = driver.find_element(By.XPATH, "//span[text()=' ']")
checkbox = element.find_element(By.XPATH, "./preceding-sibling::mat-pseudo-checkbox")
checkbox.click()

x_offset = 100
y_offset = 200
actions.move_by_offset(x_offset, y_offset).click().perform()

# Busca e extração de dados da tabela
botao_buscar = driver.find_element(By.XPATH, "//button[span[text()=' ']]")
driver.execute_script("arguments[0].scrollIntoView(true);", botao_buscar)
botao_buscar.click()

# Inicializa uma lista para armazenar os dados
data_list = []

try:
    while True:
        # Refresca a lista de linhas da tabela para garantir que a versão mais recente seja usada
        try:
            tabela_linhas = driver.find_elements(By.CSS_SELECTOR, "table.mat-table tr.mat-row")

            for row in tabela_linhas:
                try:
                    # Extraia os valores de "Veículo" e "Alerta"
                    veiculo = row.find_element(By.CSS_SELECTOR, "td.mat-column").text
                    alerta = row.find_element(By.CSS_SELECTOR, "td.mat-column").text

                    # Adiciona ao DataFrame se o "Veículo" for numérico
                    if veiculo.isdigit():
                        data_list.append({"Veículo": veiculo, "Alerta": alerta})

                except StaleElementReferenceException:
                    # Tente obter o elemento novamente em caso de erro
                    tabela_linhas = driver.find_elements(By.CSS_SELECTOR, "table.mat-table tr.mat-row")
                    continue

        except StaleElementReferenceException:
            # Se toda a tabela se tornar obsoleta, tente novamente do início
            continue

        # Converte a lista em DataFrame
        df = pd.DataFrame(data_list)

        # Cria o DataFrame filtrado e o restante
        placa = list(range())
        df['Veículo'] = pd.to_numeric(df['Veículo'], errors='coerce')
        df_filtrado = df[df['Veículo'].isin(placa)]
        df_restantes = df[~df['Veículo'].isin(placa)]

        # Opcional: Salve os DataFrames em arquivos ou realize outras operações
        df_filtrado.to_csv('df_filtrado.csv', index=False)
        df_restantes.to_csv('df_restantes.csv', index=False)

        # Envia as mensagens para os grupos do WhatsApp
        grupo1 = ""  # Grupo 1
        grupo2 = ""  # Grupo 2

        # Função para enviar mensagens para um grupo
        def enviar_mensagem(grupo_link, mensagem):
            try:
                # Envia a mensagem
                kit.sendwhatmsg_to_group_instantly(grupo_link, mensagem, 15, tab_close=True, close_time=3)
                print(f"Mensagem enviada para {grupo_link}")
            except Exception as e:
                print(f"Erro ao enviar mensagem para {grupo_link}: {e}")

        # Itera sobre o DataFrame e envia os avisos para os grupos
        for index, row in df.iterrows():
            mensagem = f"Veículo: {row['Veículo']}\nAlerta: {row['Alerta']}"
            
            # Envia a mensagem para o grupo 1
            enviar_mensagem(grupo1, mensagem)
            
            # Espera um pouco para evitar bloqueios no WhatsApp
            time.sleep(5)
            
            # Envia a mensagem para o grupo 2
            enviar_mensagem(grupo2, mensagem)
            
            # Espera novamente para evitar bloqueios
            time.sleep(5)


except KeyboardInterrupt:
    print("Interrompido pelo usuário")

finally:
    # Fechar o navegador
    driver.quit()

# Exiba o DataFrame
print(df)


In [ ]:
import pywhatkit as kit

# Substitua pelo número de telefone do contato
phone_number = "+557193643490"  # Inclua o código do país (por exemplo, +55 para o Brasil)
message = "Oi, como você está?"

# Envia a mensagem para o contato imediatamente
kit.sendwhatmsg_instantly(phone_no=phone_number, message=message, tab_close=False)

print("Mensagem enviada com sucesso!")
